## 決算プロのサイトから決算スケジュールを取得するコード

In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import time
import sys
sys.path.append('../')

In [2]:
#決算発表情報から四半期決算短信のデータだけに絞る
def filtering(_df):
    _df = _df[_df['content'].str.contains('決算')].sort_values('time')
    _df = _df[_df['content'].str.contains('年')].sort_values('time')
    _df = _df[_df['content'].str.contains('月')].sort_values('time')
    _df = _df[~_df['content'].str.contains('説')] #説明資料を省く
    _df = _df[~_df['content'].str.contains('修正')] #修正資料を省く
    _df = _df[~_df['content'].str.contains('訂正')] #訂正資料を省く
    _df = _df[~_df['content'].str.contains('補')] #補足資料を省く
    _df = _df[~_df['content'].str.contains('参考')] #参考資料を省く
    _df = _df[~_df['content'].str.contains('過年度')] #過年度の資料を省く
    _df = _df[~_df['content'].str.contains('概要')] #決算概要の資料を省く
    _df = _df[~_df['content'].str.contains('資料')] #資料を省く
    _df = _df[~_df['content'].str.contains('プレゼン')] #プレゼンテーションを省く
    _df = _df[~_df['content'].str.contains('連結')] #連結決算を省く
    return _df

In [3]:
#該当の日付の決算発表情報を取得
def get_kessan_calendar(_date):
    '''
    _date = 'YYYYMMDD'
     '20190914'：決算に関する発表がない。決算データ無。
     '20190921'：決算以外の発表がある日。決算データ無。
     '20190923'：休場日。決算データ無。
     '20190926'：通常日。決算データ有。8件の決算発表有。
    '''
    url = 'http://ke.kabupro.jp/hist/{}.htm'.format(_date)
    soup = BeautifulSoup(requests.get(url).content,'html.parser')
    tag_div = soup.find_all('div',id='centercont')[0]
    tag_table = tag_div.find_all('table')[0]
    tag_a = tag_table.find_all('a')

    try:
        _data = np.array([d.text for d in tag_a]).reshape(-1,4)
        _df = pd.DataFrame(_data, columns = ['code', 'name', 'content', 'time'])
        _df['date'] = [datetime.strptime(_date, '%Y%m%d')]*_data.shape[0]
        _df = filtering(_df)
        if len(_df) == 0:
            print(_date, '決算に関する発表無し。（業績に関する発表有り）')
            return None
        else:
            return _df
    except (ValueError):
        print(_date, '休場日')
        return None
    except (AttributeError):
        print(_date, '該当するデータはありません')
        return None

In [4]:
from_ = '20180101'
to_ = '20190926'

from_date = datetime.strptime(from_, '%Y%m%d')
to_date = datetime.strptime(to_, '%Y%m%d')
lookback = (to_date - from_date).days

date = to_date
dfs = []
for i in tqdm(range(lookback+1)):
    str_date = datetime.strftime(date, '%Y%m%d')
    df = get_kessan_calendar(str_date)
    if df is not None:
        dfs.append(df)
    date = date - timedelta(days=1)

  1%|          | 5/634 [00:01<02:07,  4.95it/s]

20190923 休場日
20190922 休場日


  1%|          | 6/634 [00:01<01:56,  5.37it/s]

20190921 決算に関する発表無し。（業績に関する発表有り）


  1%|▏         | 9/634 [00:01<02:02,  5.09it/s]

...

20180106 該当するデータはありません


100%|█████████▉| 633/634 [02:40<00:00,  3.94it/s]

20180103 休場日
20180102 休場日


100%|██████████| 634/634 [02:40<00:00,  3.94it/s]

20180101 休場日


In [5]:
df = pd.concat(dfs).reset_index(drop = True)

In [6]:
# df.to_csv('kessan_calendar_20180101-20190926.csv')

In [47]:
df = pd.read_csv('kessan_calendar_20180101-20190926.csv')

In [48]:
quarter = []
fy = []
for i,cont in enumerate(df['content']):
    if cont.find('半期') != -1:
        quarter.append(cont[cont.find('第'):cont.find('半期')+2])
        fy.append(cont[:cont.find('月')+1])
    elif cont.find('決算短') != -1:
        quarter.append(cont[:cont.find('決算短')])
        fy.append(cont[:cont.find('月')+1])
    else:
        quarter.append('-')
        fy.append('-')        
df['quarter'] = quarter
df['fy'] = fy
#filteringで省ききれなかったものを削除
df = df[df['quarter']!='-']

In [51]:
for fy in df['fy']:
    y, m = fy[:-1].split('年')
    if y.find('令和')!=-1:
        y.
    print(y,m)

2020 ５
2019 11
2019 11
2020 ５
2019 ８
2020 ５
令和2 2
2020 ２
2020 2
2020 5
2020 ２
2020 2
2020 ２
2020 ５
2020 3
令和元 10
2020 5
2019 ７
2020 ４
2020 ２
2019 ７
2019 10
2020 １
2019 ７
2019 10
2019 10
2020 ４
2019 10
2020 １
2020 ４
2019 ７
2020 ４
2020 １
2019 ７
2020 1
2020 ４
2020 ４
2020 1
2019 10
2019 ７
2019 ７
2020 1
2020 １
2019 ７
2020 ４
2019 ７
2019 10
2019 ７
2019 10
2020 １
2019 10
2019 10
2019 ７
2020 ４
2019 ７
2020 4
2019 10
令和２ １
2019 12
2019 ７
令和元 ７
2020 1
2019 12
2019 ７
2020 ４
2019 ９
2019 10
2019 10
令和２ １
2019 7
2019 7
2019 7
2020 4
2019 7
2020 １
2020 １
2019 ７
2020 ４
2019 10
2019 ７
2020 ５
2019 10
2019 ７
2019 ７
令和２ １
2019 ７
2020 １
2020 ４
2019 ７
2020 １
2020 １
令和2 4
2020 ５
2019 10
2019 ７
2020 4
2020 １
2019 7
2019 ７
2019 10
令和２ １
2020 ４
2019 10
2019 12
2019 10
2020 4
2020 1
2020 4
令和元 10
2020 1
令和２ １
2019 10
2019 10
2020 5
2019 10
2020 ４
2020 1
2020 4
2020 ４
2020 ４
2020 １
2020 １
2020 ４
2019 10
2019 10
2020 ４
2020 ４
2020 １
2020 1
2020 4
令和元 ７
2019 10
2020 １
2019 10
2019 10
2020 １
2020 ４
令和２ １
2019 10
2019 

2020 ３
2020 ３
2019 12
2020 3
2020 ３
2020 ３
2020 3
2020 ３
2019 12
2020 ３
2020 3
2020 ３
2019 6
2019 12
2020 ３
2020 ３
2020 ３
2019 12
2019 12
2020 3
2019 12
2020 ３
2020 ３
2019 12
2020 3
2019 ９
2020 ３
2019 12
2020 ３
2020 ３
2020 ３
2019 ６
2019 12
2019 12
2020 3
2020 ３
2020 3
2020 3
2019 12
2020 ３
2019 12
2019 12
2020 ３
2020 3
2019 12
2020 3
2020 ３
2019 ９
2020 3
2019 12
2020 3
2019 12
2020 ３
2019 12
2019 12
2020 3
2019 9
2020 3
2020 ３
2020 3
2020 ３
2019 ６
２０１９ １２
2019 ６
2020 ３
2020 ３
２０１９ １２
2019 12
2019 12
2020 3
令和2 3
2020 ３
2020 ３
2019 ６
2020 3
2020 3
2019 12
2020 ３
2020 3
2019 12
2020 3
2020 ３
2020 ３
2020 ３
2020 ３
2020 3
2020 ３
2019 ６
2020 3
2020 3
2020 ３
2019 12
2019 12
2020 3
2020 ３
2019 12
2019 12
2020 3
2020 3
2019 ６
2019 9
2020 3
2020 3
令和２ ３
2020 3
2020 ３
2020 ３
2020 ３
2020 3
2020 ３
2020 ３
2020 3
2019 12
2020 ３
2020 3
2020 ３
2019 12
2020 ３
2020 ３
令和２ ３
2019 ６
2019 ６
2019 12
2020 3
2020 3
2020 ３
2020 3
2020 ３
2020 ３
2019 9
2019 12
2019 6
2020 3
2020 3
2020 3
2019 12
2020 ３
2020 ２
2020

2020 3
2020 ３
２０２０ ３
2020 ３
2020 ３
2019 12
2020 ３
2020 ３
2020 ３
2020 3
2019 12
2019 12
2020 ３
2020 3
2020 3
2019 9
2020 ３
2020 3
2020 ３
2020 3
2020 2
2020 ３
2020 ２
令和元 ８
2019 ５
2020 ２
2019 11
2019 ８
令和2 2
2020 3
2019 ５
2019 12
2019 12
2020 2
令和元 11
2019 8
2020 ２
2020 ２
2019 ５
2020 ２
2020 2
2019 5
2019 11
令和元 ５
2019 ５
2020 ２
2020 2
2020 度２
2019 ５
2020 ２
2019 ８
2019 5
2020 ２
2019 11
2019 ５
2019 11
2019 ５
2019 ５
令和元 ５
2019 ５
2019 8
2020 ２
2019 12
2020 2
令和元 ５
2019 ５
2019 5
2019 ５
2020 2
2019 11
2020 ２
2020 ２
2019 12
2019 8
2019 ５
2020 ２
2019 ５
2019 11
2020 2
2020 ２
2020 ２
2020 ２
2020 ２
2019 ５
令和元 ８
2020 2
令和２ ２
2019 5
2020 ２
2020 ２
2019 ５
令和２ ２
2019 5
2020 ２
2020 ２
2019 ５
2019 11
2020 ２
2019 11
2019 ５
2019 11
2020 2
2019 11
2019 11
令和２ ２
２０２０ ２
２０２０ ２
2020 ２
2019 ８
2019 11
2019 5
2019 ５
2020 ２
2020 ２
2019 ８
2019 ８
2020 ２
2019 11
2020 ２
2020 ２
令和2 2
令和元 8
2019 ８
2019 8
2020 ２
2019 ８
2019 ５
2020 ２
2019 8
2020 ２
2020 ２
2019 ８
2020 2
2019 ５
2019 11
2019 ５
2020 ２
2019 5
2020 ２
2019 ８
2019 11
2

2019 ３
2019 ３
2019 12
2019 ３
平成31 ３
2019 3
2019 ３
2019 ９
2019 9
2019 3
2019 3
平成31 ３
2019 ３
平成31 ３
2019 3
2019 ３
2019 ３
2019 ３
2019 ３
2019 ９
2019 ３
2019 3
2019 ６
2019 ６
2019 12
2019 9
2019 3
2019 3
2019 12
平成31 ３
2019 12
2019 ３
2019 ３
2019 ３
2019 ３
2019 3
2019 12
2019 3
2019 ３
2019 ３
2019 12
2019 3
2019 ３
平成31 3
2019 12
2019 ３
2019 ３
平成31 ３
平成31 3
2019 ９
2019 3
2019 ３
2019 12
2019 ３
2019 ３
2019 ３
2019 ３
2019 ３
2019 3
2019 ３
2019 3
2019 ３
2019 ３
2019 ３
2019 12
2019 3
2019 ３
平成31 3
2019 ３
2019 3
2019 3
2019 3
2019 ３
2019 12
平成31 3
2019 3
2019 ６
2019 ６
2019 12
2019 12
2019 ３
2019 ３
2019 ３
2019 ３
2019 3
平成31 ３
2019 3
2019 ３
2019 ３
2019 ３
2019 3
2019 12
2019 12
2019 ３
令和元 12
2019 3
2019 3
令和元 12
2019 3
2019 3
2019 3
2019 ３
2019 3
2019 3
2019 3
2019 ３
2019 ３
2019 3
平成31 ３
2019 3
2019 ３
2019 3
平成31 ３
2019 ３
2019 3
2019 3
2019 ３
2019 12
令和元 12
2019 ３
2019 ３
2019 3
2019 ３
2019 3
2019 ３
平成31 ３
2019 12
2019 12
2019 3
2019 ３
2019 ３
2019 ３
2019 3
2019 3
2019 3
2019 3
2019 3
2019 ３
2019 3
2019 3
201

2019 12
2019 3
2019 12
2019 3
2019 ３
2019 12
2019 3
2019 3
2019 12
2019 ３
2019 3
2019 12
2019 12
2019 12
2019 ３
2019 12
2019 3
2019 12
2019 ３
2019 12
2019 12
2019 ３
2019 ３
2019 3
2019 ９
2019 ３
2019 3
2019 ３
2019 3
2019 3
2019 ３
2019 3
2019 ３
2019 ３
2019 3
2019 ３
2019 ３
2019 ３
2019 3
2019 3
2019 12
2019 12
2019 12
2019 12
2019 12
2019 12
2019 3
2019 ３
2019 3
2019 3
2019 3
平成31 3
2019 3
2019 12
2019 ３
2019 ３
2019 ３
2019 ３
平成31 3
2019 3
2019 3
2019 ３
2019 ３
平成31 ３
2019 ３
2019 ３
2019 3
2019 ３
2019 度3
2019 ３
2019 ３
2019 ３
2019 ３
2019 ３
2019 ３
2019 ３
2019 3
2019 3
2019 3
2019 ３
2019 12
2019 3
2019 12
2019 3
2019 3
2019 ３
2019 ９
2019 ３
2019 ９
2019 ３
2019 12
2019 ３
2019 3
2019 3
2019 ３
2019 3
2019 ３
2019 3
2019 ３
2019 12
2019 12
2019 3
2019 3
2019 12
2019 3
2019 3
2019 3
2019 3
2019 ３
2019 12
2019 ３
2019 ３
2019 12
2019 ３
平成31 12
2019 ３
2019 3
2019 ３
2019 ３
2019 12
2019 12
2019 3
2019 ３
2019 3
2019 12
2019 ３
2019 3
2019 9
2019 12
2019 ５
2019 12
2019 6
2019 12
2019 12
2019 3
平成31 ３
2019 3
2019 ３

...

2019 ３
2019 ３
平成31 ３
平成30 ９
2019 3
平成30 12
平成30 12
2018 12
平成30 12
2018 12
2019 3
平成30 ９
平成30 12
平成30 12
2019 ３
平成31 3
平成31 ３
平成30 12
平成31 3
平成31 3
2018 12
平成31 3
平成31 3
2019 ３
平成31 ６
平成30 12
平成30 12
平成31 3
平成30 12
平成31 ３
平成31 3
平成31 3
平成30 12
2018 ９
平成31 ３
平成31 ３
平成30 12
平成31 3
平成31 3
平成30 12
2019 3
平成30 12
2018 9
平成30 12
平成31 ３
平成31 6
2018 12
平成30 12
平成31 3
平成30 ９
2019 ３
平成30 ９
平成31 3
2018 12
平成31 3
平成31 ３
平成31 ３
平成30 12
2018 9
平成30 ９
2019 ３
2019 3
平成30 12
平成30 9
平成31 ３
2018 12
平成31 3
平成31 3
平成31 ３
平成31 3
平成31 ３
2019 ６
平成31 ３
2019 ６
平成31 ３
平成31 ３
平成31 ３
平成31 ６
2019 6
2019 ６
平成31 ６
平成31 ６
平成31 ３
平成31 ３
2019 ６
平成31 3
平成31 ３
平成31 3
平成31 ３
2019 ３
平成31 ３
平成31 ３
平成31 ３
平成30 ９
2019 3
2019 ３
平成31 ３
2019 ３
平成31 3
平成30 12
平成31 3
平成31 ６
2019 ３
2018 12
2018 12
平成30 12
平成31 3
2019 ３
2019 ３
2019 3
平成31 ３
平成31 ３
平成31 3
平成31 ３
平成31 ３
平成31 3
平成30 12
平成31 ３
平成31 3
平成31 3
2019 ３
平成30 12
平成30 ９
平成31 ３
２０１９ ３
平成31 3
平成31 ３
2019 ３
2019 ３
2019 3
平成31 ３
2019 ３
2018 12
2019 ３
平成31 ３
平成30 12
2019 ６
平成31 ３
平成3

平成31 ３
平成31 ３
平成31 ３
平成31 ３
平成31 3
2019 3
平成31 ３
平成31 ３
2019 ３
平成31 3
2019 ３
平成31 3
平成31 3
平成31 3
平成30 12
平成31 ３
平成31 3
平成31 3
平成31 3
平成31 3
平成31 3
2019 ３
平成31 3
2019 ３
平成31 3
2019 ３
平成31 ３
平成31 3
平成30 12
2019 ３
2019 ３
2019 ３
2019 3
2019 ３
平成31 ３
平成31 ３
平成31 ３
2019 ３
2018 12
2019 3
平成30 12
2019 3
2019 ３
2019 ３
2018 12
2019 ３
2018 12
平成30 12
2019 ６
平成30 12
平成31 ３
2019 6
平成31 3
平成30 12
2018 12
平成30 12
平成30 12
平成31 3
2019 3
平成31 ３
平成31 ３
2019 ３
2018 12
2019 6
平成31 3
平成31 ３
2019 ３
平成31 6
2018 9
平成30 12
平成31 3
2018 12
平成31 ３
2019 ３
平成31 3
2019 ３
平成31 ３
平成31 3
平成31 ３
平成31 ３
平成31 3
平成31 3
平成31 3
平成30 12
平成30 12
平成30 12
平成30 12
2019 3
平成31 3
平成30 12
平成31 ３
平成31 ３
平成31 3
平成30 12
2019 ３
平成31 ３
2019 3
2019 3
平成31 3
2019 ３
2019 3
２０１９ ３
平成31 ３
平成31 ３
2019 ３
2019 ３
2019 ３
2019 ３
2019 3
2019 3
平成31 ３
平成31 ３
平成31 ３
2019 ３
2019 ３
平成31 3
平成31 3
平成31 3
平成31 3
平成31 3
平成31 ３
平成31 3
平成31 ３
2019 3
2019 ３
2019 3
平成31 3
2019 ３
平成31 3
2019 3
平成31 3
平成31 ３
平成31 ３
2019 3
平成31 3
平成31 3
2019 ３
平成31 ３
平成31 ３
平成31 ３

平成31 １
平成30 10
平成30 10
2018 12
平成31 ４
2018 12
2018 12
平成30 6
平成30 6
平成30 12
平成30 6
平成30 6
平成30 6
平成30 6
2018 12
平成30 11
平成31 3
平成31 3
平成31 ３
平成31 3
平成31 ３
平成30 12
平成31 ３
平成30 9
平成30 12
平成３１ ３
2018 12
平成31 ３
2019 ３
平成30 9
2019 ３
平成30 12
2019 ３
2018 6
2018 ９
平成30 12
2019 3
2018 12
平成30 12
平成30 ６
平成31 ３
平成30 ９
平成31 ３
平成30 ９
2019 ３
平成30 ６
平成３１ ３
平成30 ６
平成30 ９
平成30 ６
平成30 9
2019 ３
平成30 ９
平成30 12
平成30 ６
平成３１ ３
平成31 ３
平成31 ３
平成30 12
平成30 12
平成30 12
平成30 12
平成31 3
平成30 12
平成30 12
平成30 12
平成30 12
2018 12
2018 6
平成30 12
2019 ３
2019 3
平成31 ３
平成30 9
平成30 9
平成31 ３
平成31 ３
平成31 ３
平成31 3
平成30 12
平成30 ９
平成30 12
2019 ３
2019 ３
2019 3
平成30 ６
平成31 ３
平成30 12
平成30 ９
平成31 3
平成30 12
2018 6
平成31 ３
2018 12
平成30 ９
平成31 3
2019 ３
2018 12
平成30 ６
平成31 ３
平成31 3
平成31 ３
平成31 ３
平成30 ９
2019 ３
平成30 12
平成31 ３
平成30 12
平成30 12
平成30 12
平成30 12
平成30 12
平成31 ３
平成31 ３
平成30 6
平成30 9
平成31 ３
平成31 3
平成30 12
平成30 12
平成31 ３
平成30 6
平成30 12
平成30 12
2018 12
平成31 3
2018 12
平成31 ３
平成30 12
2019 ３
2018 12
平成31 ３
平成30 ６
2019 ３
平成30 ６
平成30 ６
平成

2019 ３
2019 ３
2018 ３
2019 ３
2019 3
平成31 3
平成31 3
平成31 3
平成31 ３
2019 ３
2018 6
平成31 ３
2019 3
平成31 3
2019 3
2019 ３


ValueError: too many values to unpack (expected 2)

In [ ]:
from database.models import KabuDatabase
DB = KabuDatabase()

In [ ]:
DB.get_data('gyoseki')

In [ ]:
df